In [ ]:
import csv

from time import process_time
from math import pi, radians, degrees

import numpy as np
import pandas as pd

from matplotlib.pyplot import figure, subplots, show
from astropy.coordinates import SkyCoord
import astropy.units as u

MAKE_ONLINE_QUERIES = True

Database Names:
```
H_Au06_L3_Angle030_Ext
H_Au16_L3_Angle030_Ext
H_Au21_L3_Angle030_Ext
H_Au23_L3_Angle030_Ext
H_Au24_L3_Angle030_Ext
H_Au27_L3_Angle030_Ext

I_Au06_L3_Angle030_Ext
I_Au16_L3_Angle030_Ext
I_Au21_L3_Angle030_Ext
I_Au23_L3_Angle030_Ext
I_Au24_L3_Angle030_Ext
I_Au27_L3_Angle030_Ext

I_Au06_L3_Angle030_NoExt
I_Au16_L3_Angle030_NoExt
I_Au21_L3_Angle030_NoExt
I_Au23_L3_Angle030_NoExt
I_Au24_L3_Angle030_NoExt
I_Au27_L3_Angle030_NoExt 
```

## Request Utilities

In [ ]:
# Here a more modern interface to VirgoDB
import re

import h5py
import requests

from io import StringIO

VIRGO_DB_BASE_URL = "http://virgodb.dur.ac.uk:8080/MyMillennium"

CHUNK_SIZE = 8192
LINE_CHUNK_SIZE = 1048

# Mapping between SQL and numpy types
numpy_dtype = {
    "real": np.float32,
    "float": np.float64,
    "int": np.int32,
    "bigint": np.int64,
    "char": np.dtype("|S256"),
    "nvarchar": np.dtype("|S256"),
    "decimal": np.float64
}

In [ ]:
class VirgoDBError(Exception):
    """Base class for exceptions raised by this module"""
    pass

class SQLError(VirgoDBError):
    """Exception raised if an SQL query fails"""
    pass

class BadResponseError(VirgoDBError):
    """Exception raised if we can't interpret the result of a query"""
    pass

In [ ]:
from itertools import dropwhile


def read_column_info(lines_gen):
    """
    Parse the first few lines of the returned CSV file. All the 'metadata' is
    prefixed with a #. The generator consumes all the # lines and the first line
    with no # sign (should be the comma separated names again)
    """
    # Skip rows until we reach QUERYTIMEOUT
    lines_gen = dropwhile(lambda l: not l.startswith("#QUERYTIMEOUT"), lines_gen)

    # Skip QUERYTIMEOUT & QUERYTIME
    if not(next(lines_gen).startswith("#QUERYTIMEOUT")):
        raise BadResponseError("Don't understand result header!")
        
    if not(next(lines_gen).startswith("#QUERYTIME")):
        raise BadResponseError("Don't understand result header!")

    # Read column info
    # (also discards line with full list of column names)
    columns = []
    while True:
        line = next(lines_gen)
        if line[0] != "#":
            break
        else:
            m = re.match("^#COLUMN ([0-9]+) name=([\w]+) JDBC_TYPE=(-?[0-9]+) JDBC_TYPENAME=([\w]+)$", line)
            if m is not None:
                columns.append(m.groups())
            else:
                raise BadResponseError("Don't understand column info: "+line)

    # Turn parsed column info into meaningful quantities
    names = [col[1] for col in columns]
    dtypes = [numpy_dtype[col[3]] for col in columns]
    rectype = np.dtype([(col[1], numpy_dtype[col[3]]) for col in columns])
    return names, dtypes, rectype

In [ ]:
def stream_query_hdf5(query: str, filename: str, conn: requests.Session, force=False, verbose=False):
    """
    Makes a query to the Millenium database and streams it to a new HDF5 file.
    Code is adapted from the VirgoDB client:
        http://virgodb.dur.ac.uk:8080/MyMillennium/pages/help/python/virgodb.py
    """

    if not MAKE_ONLINE_QUERIES and not force:
        print("Making queries disabled")
        return filename, None
    
    payload = {
        'action': 'doQuery',
        'SQL': query,
    }
    
    start = process_time()
    
    with conn.get(VIRGO_DB_BASE_URL, params=payload, stream=True) as r:
        # Throw exception if there is some error
        r.raise_for_status()

        lines = r.iter_lines(chunk_size=CHUNK_SIZE, decode_unicode=True)

        # Read column info
        names, dtypes, rectype = read_column_info(lines)

        if verbose:
            print("Column info:")
            for i, (n, d) in enumerate(zip(names, dtypes)):
                print(f"{n}  {d}  {rectype[i]}")
        # Create the output file and datasets
        with h5py.File(filename, "w") as out:
            for name, dtype in zip(names, dtypes):
                out.create_dataset(name,
                                   dtype=dtype,
                                   shape=(0, ),
                                   maxshape=(None, ),
                                   chunks=(LINE_CHUNK_SIZE, ))

            # Iterate over the remaining lines & put them in the hdf5 file
            read_lines = "\n".join(l for _, l in zip(range(LINE_CHUNK_SIZE), lines))
            nwritten = 0
            while read_lines != "":

                # Turn the read lines into a list of tuples for the data
                f = StringIO(read_lines)
                dtypes = {name: rectype.fields[name][0] for name in rectype.names}
                data = pd.read_csv(f,
                                   names=rectype.names,
                                   dtype=dtypes,
                                   delimiter=",",
                                   engine="c").to_records(index=False)

                # Insert into the HDF5 file
                for name in names:
                    dataset = out[name]
                    dataset.resize((nwritten+data.shape[0],))
                    dataset[nwritten:nwritten+data.shape[0]] = data[name]

                nwritten += data.shape[0]

                read_lines = "\n".join(l for _, l in zip(range(LINE_CHUNK_SIZE), lines))

    end = process_time()
    duration = end - start
    print(f"Query download took {duration:.4f}s")
    
    return filename

In [ ]:
def stream_query(query: str, filename: str, conn: requests.Session, force=False):
    # Inspiration taken from SO answer:
    # https://stackoverflow.com/questions/16694907/download-large-file-in-python-with-requests

    if not MAKE_ONLINE_QUERIES and not force:
        print("Making queries disabled")
        return filename, None

    payload = {
        'action': 'doQuery',
        'SQL': query,
    }
    # TODO: Assert that the file exists and is empty

    # Make query
    headers = None
    start = process_time()
    
    with conn.get(VIRGO_DB_BASE_URL, params=payload, stream=True) as r:
        r.raise_for_status()
        headers = r.headers
        with open(filename, 'wb') as f:
            for chunk in r.iter_content(chunk_size=CHUNK_SIZE):
                f.write(chunk)
                
    end = process_time()
    duration = end - start
    print(f"Query download took {duration:.4f}s")
    
    return filename, headers

In [ ]:
# Initialize session
conn = requests.Session()

# TODO: move to file or input
username = "USERNAME"
password = "PASSWORD"
conn.auth = (username, password)

In [ ]:
import vaex

query = """
select top 10000 RA, Dec, ParticleId
from  Grand2018a..H_Au06_L3_Angle030_Ext 
"""

outfile = "tmp.hdf5"
stream_query_hdf5(query, outfile, conn, force=True)

df = vaex.open(outfile)
df

## Coordinate stuffs
\begin{align*}
\sin b &= sin\delta_{NGP}\sin\delta + cos\delta_{NGP}\cos(\alpha - \alpha_{NGP})\\
\cos b \sin(l_{NCP} - l) &= \cos\delta\sin(\alpha- \alpha_{NGP})\\
\cos b \cos(l_{NCP} - l) &= \cos\delta_{NGP}\sin\delta - \sin\delta_{NGP}\cos\delta\cos(\alpha- \alpha_{NGP})\\
\end{align*}

In [ ]:
df = df.to_pandas_df()

In [ ]:
# Standard coordinates
NGP = SkyCoord(0, 90, unit=u.deg, frame='galactic').icrs
RA_NGP = NGP.ra.to(u.rad).value
DEC_NGP = NGP.dec.to(u.rad).value

NCP = SkyCoord(0, 90, unit=u.deg, frame='icrs').galactic
l_NCP = NCP.l.to(u.rad).value


# Some sample coordinates
RA = df['RA'][:10]
Dec = df['Dec'][:10]


# WORKS: 
b = np.arcsin(np.sin(DEC_NGP)*np.sin(Dec) + np.cos(DEC_NGP)*np.cos(Dec)*np.cos(RA - RA_NGP))
b_str = f"asin(sin({DEC_NGP})*sin(Dec) + cos({DEC_NGP})*cos(Dec)*cos(RA - {RA_NGP})) as b"

# Seems to work:
l = l_NCP - np.arctan2( np.cos(Dec)*np.sin(RA - RA_NGP) ,
                       np.cos(DEC_NGP)*np.sin(Dec) - np.sin(DEC_NGP)*np.cos(Dec)*np.cos(RA - RA_NGP) )
l_str = f"{l_NCP} - atn2(cos(Dec)*sin(RA - {RA_NGP}) , cos({DEC_NGP})*sin(Dec) - sin({DEC_NGP})*cos(Dec)*cos(RA - {RA_NGP}) ) as l"

print('-----------')

c = SkyCoord(RA, Dec, unit=u.rad, frame='icrs').galactic
ref_b = c.b.to(u.rad).value
print(ref_b - b)

print('-----------')

ref_l = c.l.to(u.rad).value
print(ref_l - l)

print('-----------')
print(b_str)
print()
print(l_str)
print()

## Box Query

In [ ]:
general_query = """
SELECT *    
    FROM (
        SELECT *, 
        CASE WHEN l_ > PI() THEN l_ - 2*PI() ELSE l_ END as l,
        CASE WHEN b_ > PI() THEN b_ - 2*PI() ELSE b_ END as b
        FROM (
            SELECT
                *,
                asin(sin(0.4734773249532946)*sin(Dec) + cos(0.4734773249532946)*cos(Dec)*cos(RA - 3.366032882941064)) as b_,
                2.145566851522591 - atn2( cos(Dec)*sin(RA - 3.366032882941064) , cos(0.4734773249532946)*sin(Dec) - sin(0.4734773249532946)*cos(Dec)*cos(RA - 3.366032882941064) ) as l_
            FROM  {table_name}
            WHERE     
                    ((Gmagnitude between 13.5 and 16.5) OR (GmagnitudeObs between 13.5 and 16.5))
                AND ((parallax - parallaxerror < 0.00025) OR (parallaxobs - parallaxerror < 0.00025))

            ) as tbl1
        ) as tbl2

WHERE 
    l between {lower} and {upper} AND
    b between {lower} and {upper}
"""

In [ ]:
lim = radians(12)
query = general_query.format(table_name='Grand2018a..H_Au06_L3_Angle030_Ext', lower=-lim, upper=lim)
print(query)

In [ ]:
box_query = general_query.format(table_name='Grand2018a..H_Au06_L3_Angle030_Ext', lower=-lim, upper=lim)
box_file = './data/H_Au06_Ext.hdf5'
stream_query_hdf5(box_query, box_file, conn)

In [ ]:
box_file = './data/I_Au06_Ext.hdf5'
box_query = general_query.format(table_name='Grand2018a..I_Au06_L3_Angle030_Ext', lower=-lim, upper=lim)
stream_query_hdf5(box_query, box_file, conn)

**NOTE: this times out after some time. Not sure why, because the query should be larger if possible.**

In [ ]:
box_file = './data/I_Au06_NoExt.csv'
box_file_hdf5 = './data/I_Au06_NoExt.hdf5'
box_query = general_query.format(table_name='Grand2018a..I_Au06_L3_Angle030_NoExt', lower=-lim, upper=lim)
stream_query(box_query, box_file, conn)

In [ ]:
# Old reading from the CSV file. Do this once, and convert it to a HDF5
box_file_hdf5 = './data/I_Au06_NoExt.hdf5'
df = vaex.from_csv(box_file, convert=box_file_hdf5, comment='#')

Idea, make a list of all the stars is the selection and query their IDs. This should result in a lot less stars in the query.

```python
in_query = """
SELECT * FROM Grand2018a..I_Au06_L3_Angle030_NoExt
WHERE StarID in (
    {}
)
""".format()
```

## Determine the limits:

In [ ]:
import csv
pointings_fn = './data/fieldcenters_spec.csv'
select = 2  # control the number of pointings

def read_pointings(filename):
    ls = []
    bs = []
    with open(filename, 'r') as f:
        reader = list(csv.reader(f))
        for line in reader[1:]:
            l, b = line
            ls.append(float(l.strip()))
            bs.append(float(b.strip()))

    ls = np.array(ls)
    bs = np.array(bs)
    return ls, bs

ls, bs = read_pointings(pointings_fn)
ls = np.where(ls > 180, ls - 360, ls)
bs = np.where(bs > 180, bs - 360, bs)
print(ls)
print(bs)

pointings = SkyCoord(ls, bs, unit=u.deg, frame='galactic')
pointings

In [ ]:
lower = min(ls.min(), bs.min()) - 1
print(lower)

upper = max(ls.max(), bs.max()) - 1
print(upper)

print(ls.min(), ls.max())
print(bs.min(), bs.max())

Conclusion: Easiest is just to query the 12x12 area.

count: 207293135 (parallax - parallaxerror) > 0.00025)
count: 597414673 (parallax - parallaxerror < 0.00025)
count: 150032778 parallaxobs < 0!!!!
count: 654675042 parallaxobs > 0
